# COLLISION ÉLASTIQUE DE DEUX DISQUES

In [ ]:
import urllib
import os
from notebook import notebookapp

import numpy as np
import math

from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Slider, Button, TextInput
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.events import ButtonClick

from bokeh.io import output_notebook, show, export_png

from time import sleep

import imageio

output_notebook()

In [ ]:
#Image("elastic_coll.png")

### Ensemble de fonctions pour définir divers paramètres



In [ ]:
# fonction pour calculer la masse
mass = lambda P : (4 / 3) * P['density'] * np.pi * P['radius'] ** 3

# fonction pour calculer vitesse en x et y 
vel_ = lambda P: (P['v']*np.cos(np.deg2rad(P['alpha'])), P['v']*np.sin(np.deg2rad(P['alpha'])))

# fonction pour calculer la trajectoire
traj_ = lambda P, t, dt:  [P['x'] + P['v_x']*np.linspace(0, t, t/dt)/10, P['y'] + P['v_y']*np.linspace(0, t, t/dt)/10]

# fonction pour calculer la position des disques
new_ = lambda P, dt: (P['x'] + P['v_x']*dt, P['y'] + P['v_y']*dt)

# fonction pour calculer le centre de gravité du système 
get_CG = lambda P1, P2: (P1['x']*P1['m']/(P1['m']+P2['m']) + P2['x']*P2['m']/(P1['m']+P2['m']), \
                        P1['y']*P1['m']/(P1['m']+P2['m']) + P2['y']*P2['m']/(P1['m']+P2['m']))

# fonction pour calculer la distance entre les deux disques
dist_ = lambda P1, P2: math.sqrt((P1['x'] - P2['x'])**2 + (P1['y'] - P2['y'])**2)

# calculer la projection de la vitesse sur un vecteur
proj_v = lambda P, w: (P['v_x']*w[0] + P['v_y']*w[1])/(np.linalg.norm(w))**2

# ¿?
proj_v_vec = lambda P, w: proj_v(P,w)*w

# calculer énergie cinétique
e_cinetique = lambda P1, P2: P1['m']*P1['v']**2 + P2['m']*P2['v']**2

# calculer quantité de mouvement
quant_mov_u = lambda P1, P2, u: P1['m']*proj_v(P1, u) + P2['m']*proj_v(P2,u)

# calculer vitesse après collision
new_v_coll = lambda P, w, u: (proj_v_vec(P,w) - proj_v_vec(P,u))

# fonction pour définir les têtes des flèches des vecteurs 
build_arrow_x = lambda v, a, x: np.linspace(v - 4*np.cos(np.deg2rad((-1)*x)),v, 10)
#build_arrow_x = lambda v, a, x: np.linspace(v*np.cos(np.deg2rad(a)) - 0.1*v*np.cos(np.deg2rad(a - x)),v*np.cos(np.deg2rad(a)), 10)


# v --> vx + cx (final point); a --> angle of vector; x --> omega (10º)
build_arrow_y = lambda v, a, x: np.linspace(v - 4*np.sin(np.deg2rad((-1)*x)),v, 10)
#build_arrow_y = lambda v, a, x: np.linspace(v*np.sin(np.deg2rad(a)) - 0.1*v*np.sin(np.deg2rad(a - x)),v*np.sin(np.deg2rad(a)), 10)


def get_arrows(CG):
    vx = CG['vx']
    vy = CG['vy']
    x = CG['init_x']
    y = CG['init_y']
    
    v = np.sqrt(vx**2 + vy**2)
    v_vec = np.array((vx,vy))
    v_vec_up = 4*np.array((-vy, vx)/v)
    v_vec_down = 4*np.array((vy,-vx)/v)
    
    if vx == 0 and vy == 0:
        p_up = v_vec
        p_down = v_vec
    else:
        p_up = 0.9*v_vec + v_vec_up
        p_down = 0.9*v_vec + v_vec_down
    
    a_up_x = x + np.linspace(p_up[0],v_vec[0],10)
    a_up_y = y + np.linspace(p_up[1],v_vec[1],10)
    a_down_x = x + np.linspace(p_down[0],v_vec[0],10)
    a_down_y = y + np.linspace(p_down[1],v_vec[1],10)
    
    return a_up_x, a_up_y, a_down_x, a_down_y
    


def convert_to_array(P):
        for k in ['x', 'init_x', 'y', 'init_y']:
            P[k] = np.array(P[k])

In [ ]:
## PLOT

def modify_doc(doc):
    
    # Conditions initiales 
    dt = 0.005
    T = 10
    x_a, x_b = -90, 90
    
    y_a, y_b = 0,0

    v_a, v_b = 15, 15
    
    alpha_a, alpha_b = 0, 180
    rad_a, rad_b = 15, 15
    
    # définir limits du scenario
    Y_MAX = 100
    Y_MIN = -100
    X_MAX = 200
    X_MIN = -200
    
    scenario = {'y_MAX': Y_MAX, 'y_MIN': Y_MIN, 'x_MAX': X_MAX, 'x_MIN': X_MIN}

    # Initialize
    def initialize(t, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b):
        A = {'x': x_a, 'y': y_a, 'v':v_a, 'alpha': alpha_a, 'density': 1, 'radius': rad_a, 't':t, 'name':'A'}
        B = {'x': x_b, 'y': y_b, 'v':v_b, 'alpha': alpha_b, 'density': 1, 'radius': rad_b, 't':t, 'name':'B'}

        A['v_x'], A['v_y'] = vel_(A)
        B['v_x'], B['v_y'] = vel_(B) 

        A['m'], B['m'] = mass(A), mass(B)

        CG = {}
        CG['m'] = (A['m'] + B['m'])
        CG['x'] = (1/CG['m']) * (A['x']*A['m'] + B['x']*B['m'])
        CG['y'] = (1/CG['m']) * (A['y']*A['m'] + B['y']*B['m'])
        CG['vx'] = (1/CG['m']) * (A['v_x']*A['m'] + B['v_x']*B['m'])
        CG['vy'] = (1/CG['m']) * (A['v_y']*A['m'] + B['v_y']*B['m'])
        return A,B, CG
    
    def centre_grav(P1, P2, idx=0):
        
        """ calculer le centre de gravité"""
        m_tot = (P1['radius'] + P2['radius'])
        c_x = P1['traj'][idx,0]*P1['radius']/m_tot + P2['traj'][idx,0]*P2['radius']/m_tot
        c_y = P1['traj'][idx,1]*P1['radius']/m_tot + P2['traj'][idx,1]*P2['radius']/m_tot
        return c_x, c_y
    
    def centre_grav_vel(P1, P2, idx=0):
        m_tot = (P1['radius'] + P2['radius'])
        v_x = -P1['v_x']*P1['radius']/m_tot - P2['v_x']*P2['radius']/m_tot
        v_y = P1['v_y']*P1['radius']/m_tot + P2['v_y']*P2['radius']/m_tot
        return v_x, v_y
    
    def update_position(P1, P2, CG, dt):
        """ calculer la nouvelle position"""
        P1['x'], P1['y'] = new_(P1, dt)
        P2['x'], P2['y'] = new_(P2, dt)
        CG['x'], CG['y'] = get_CG(P1, P2)


    def update_velocity(P1, P2):
        """ calculer la velocité après collision entre les deux particules"""
        # calculer vecteur perpendiculaire à la collision
        u = np.array([P2['x'] - P1['x'], P2['y'] - P1['y']])/dist_(P1,P2)

        # point collision
        p_x = P1['x'] + u[0]*P1['radius']/dist_(P1,P2)
        p_y = P1['y'] + u[1]*P1['radius']/dist_(P1,P2)

        # vecteur tangent à la collision
        w = np.array([u[1], -u[0]])

        # mettre à jour velocité après collision
        P1['v_x'], P1['v_y'] = new_v_coll(P1, w, u)
        P2['v_x'], P2['v_y'] = new_v_coll(P2, w, u)

        P1['v'] = math.sqrt(P1['v_x']**2 + P1['v_y']**2)
        P2['v'] = math.sqrt(P2['v_x']**2 + P2['v_y']**2)


    def verifier_limites(P, scenario):
        """ calculer velocité après collision entre particules et un des limites du scénario"""

        col = False # boolean pour indiquer si il y a collision ou pas 

        # verifier si il y a une collision entre P et limite superiéure
        if P['y'] >= scenario['y_MAX'] - P['radius']*1.2 : # -5 pour épaisseur de la ligne
            col = True
            u = np.array([0, -1])
            w = np.array([1, 0])


        # verifier si il y a une collision entre P et limite inferiéure
        elif P['y'] <= scenario['y_MIN'] + P['radius']*1.2 : #+3
            col = True
            u = np.array([0, 1])
            w = np.array([-1, 0])


        # verifier si il y a une collision entre P et limite droite
        elif P['x'] >= scenario['x_MAX'] - P['radius']*1.2 : #-3
            col = True
            u = np.array([-1, 0])
            w = np.array([0, 1])


        # verifier si il y a une collision entre P et limite gauche
        elif P['x'] <= scenario['x_MIN'] + P['radius']*1.2 : #+3
            col = True
            u = np.array([1, 0])
            w = np.array([0, -1])

        # si il y a eu une collision, mettre à jour la velocité 
        if col:
            P['v_x'], P['v_y'] = new_v_coll(P, w, u)
            P['v'] = math.sqrt(P['v_x']**2 + P['v_y']**2)
    
    
    def update_values_time(P1, P2, CG, dt, scenario):
        """ calcules évolution des valeurs dans le temps"""
        t = 0
        P1['traj'] = [(P1['x'], P1['y'])]
        P2['traj'] = [(P2['x'], P2['y'])]
        CG['traj'] = [(CG['x'], CG['y'])]

        P1['init_x'] = P1['x']
        P1['init_y'] = P1['y']
        P2['init_x'] = P2['x']
        P2['init_y'] = P2['y']
        CG['init_x'] = CG['x']
        CG['init_y'] = CG['y']

        while t < min(P1['t'], P2['t']):

            # calculer prochain possition
            update_position(P1, P2, CG, dt)
            
            
            P1['traj'].append((P1['x'], P1['y']))
            P2['traj'].append((P2['x'], P2['y']))
            CG['traj'].append((CG['x'], CG['y']))

            # verifier si il y a une collision entre P1 et P2
            if dist_(P1, P2) < (P1['radius'] + P2['radius']):            
                update_velocity(P1, P2)

            # verifier si il y a collision avec les limites du scenario
            verifier_limites(P1, scenario)
            verifier_limites(P2, scenario)

            t += dt  

        P1['traj'] = np.array(P1['traj'])
        P2['traj'] = np.array(P2['traj'])
        CG['traj'] = np.array(CG['traj'])
    
    
    
    
    
    # Calculer les valeurs initiales
    A, B, CG = initialize(T, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b) 
    update_values_time(A, B, CG, dt, scenario)
    
    # Créer dictionnaire pour acceder aux données
    source_A = ColumnDataSource(data = {'x':[A['init_x']], 'y':[A['init_y']], 'radius':[A['radius']]})
    source_B = ColumnDataSource(data = {'x':[B['init_x']], 'y':[B['init_y']], 'radius':[B['radius']]})
    
    source_A_trj = ColumnDataSource(data= {'x_t': A['traj'][:,0], 'y_t': A['traj'][:,1]})
    source_B_trj = ColumnDataSource(data= {'x_t': B['traj'][:,0], 'y_t': B['traj'][:,1]})
    
    source_CG = ColumnDataSource(data = {'x': [CG['init_x']], 'y': [CG['init_y']]})
    
    if CG['init_x'] + CG['vx']*5 != 0:
        alpha = np.rad2deg(np.arctan((CG['init_y'] + CG['vy']*5)/(CG['init_x'] + CG['vx']*5)))
    else: 
        alpha = np.rad2deg(np.arctan(10**9))
    source_VG = ColumnDataSource(data = {'x': np.linspace(CG['init_x'], CG['init_x'] + CG['vx']*5, 10), \
                                         'y': np.linspace(CG['init_y'], CG['init_y'] + CG['vy']*5, 10)})
    
    
    # Définir sliders 
    slider_Ax = Slider(start=-180, end=180, value=-90, step=1, title='Ax [m]:')
    slider_Bx = Slider(start=-180, end=180, value=90, step=1, title='Bx [m]:')
    
    slider_Ay = Slider(start=-80, end=80, value=0, step=1, title='Ay [m]:')
    slider_By = Slider(start=-80, end=80, value=0, step=1, title='By [m]:')
    
    slider_Ar = Slider(start=5, end=25, value=15, step=1, title='Rayon A [m]:')
    slider_Br = Slider(start=5, end=25, value=15, step=1, title='Rayon B [m]:')
    
    slider_Av = Slider(start=2, end=40, value=15, step=1, title='Vitesse A [m/s]:')
    slider_Bv = Slider(start=2, end=40, value=15, step=1, title='Vitesse [m/s]:')
    
    slider_A_alpha = Slider(start=0, end=360, value=0, step=5, title='α [º]:')
    slider_B_alpha = Slider(start=0, end=360, value=180, step=5, title='α [º]:')
    
    play = Button(label='play')
    export = Button(label='export video')
    file_name = TextInput(title='File name', value='colllision')
    
    
    
    
    # Figure
    p = figure(title="Collision", plot_height=448, plot_width=896, y_range=(scenario['y_MIN'], scenario['y_MAX']), 
               x_range=(scenario['x_MIN'], scenario['x_MAX']), 
               background_fill_color='#ffffff')
    
    p.toolbar.logo = None
    p.toolbar_location = None

    # grille
    p.ygrid.minor_grid_line_color = 'grey'
    p.ygrid.minor_grid_line_alpha = 0.3

    p.xgrid.minor_grid_line_color = 'grey'
    p.xgrid.minor_grid_line_alpha = 0.3

    # limites du scénario
    p.line(x=np.linspace(scenario['x_MIN'],scenario['x_MAX'], 3), y=[scenario['y_MIN']]*3, line_color='#000000', line_width=10)
    p.line(x=np.linspace(scenario['x_MIN'],scenario['x_MAX'], 3), y=[scenario['y_MAX']]*3, line_color='#000000', line_width=10)
    p.line(x=[scenario['x_MIN']]*3, y=np.linspace(scenario['y_MIN'],scenario['y_MAX'], 3), line_color='#000000', line_width=10)
    p.line(x=[scenario['x_MAX']]*3, y=np.linspace(scenario['y_MIN'],scenario['y_MAX'], 3), line_color='#000000', line_width=10)
    
    # Disques
    P_A = p.circle(x='x', y='y', radius='radius', source = source_A, fill_color='#e32020', line_color='#e32020', legend='A', alpha=0.8)
    P_B = p.circle(x='x', y='y', radius='radius', source = source_B, fill_color='#0ABDE3', line_color='#0ABDE3', legend='B', alpha=0.8)

    # Trajectoire
    Traj_A = p.line(x='x_t', y='y_t', source=source_A_trj, color='#e32020', line_dash='dashed', legend='trajectoire A')
    Traj_B = p.line(x='x_t', y='y_t', source=source_B_trj, color='#0ABDE3', line_dash='dashed', legend='trajectoire B')
    
    
    # centre de gravité du système de masses 
    p_CG = p.circle(x='x', y='y', source=source_CG, size=3, fill_color='#000000', line_color='#000000', legend='CG')
    
    # vecteur vitesse centre de gravité
    l_vel_CG = p.line(x=np.linspace(CG['init_x'], CG['init_x'] + CG['vx']*5, 10), \
                y=np.linspace(CG['init_y'], CG['init_y'] + CG['vy']*5, 10), \
                line_width=1, line_color='#000000', legend='vitesse CG')
    
    CG_arrow = p.triangle(x=[CG['init_x'] + CG['vx']*5], y =[CG['init_y'] + CG['vy']*5], \
                          angle=alpha, color='#000000', size=5)
    
    
    
    
    def refresh_source(attrname, old, new):
        
        l_vel_CG.visible = True
        CG_arrow.visible= True
        
        x_a = slider_Ax.value
        x_b = slider_Bx.value
        
        y_a = slider_Ay.value
        y_b = slider_By.value
        
        v_a = slider_Av.value
        v_b = slider_Bv.value
        
        alpha_a = slider_A_alpha.value
        alpha_b = slider_B_alpha.value
        
        rad_a = slider_Ar.value
        rad_b = slider_Br.value

        A, B, CG = initialize(T, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b) 
        update_values_time(A, B, CG, dt, scenario)
        
        source_A.data = {'x':[A['init_x']], 'y':[A['init_y']], 'radius':[A['radius']]}
        source_B.data = {'x':[B['init_x']], 'y':[B['init_y']], 'radius':[B['radius']]}

        source_A_trj.data = {'x_t': A['traj'][:,0], 'y_t': A['traj'][:,1]}
        source_B_trj.data = {'x_t': B['traj'][:,0], 'y_t': B['traj'][:,1]}
        
        source_CG.data = {'x': [CG['init_x']], 'y': [CG['init_y']]}
        
        if CG['init_x'] + CG['vx']*5 != 0:
            alpha = np.rad2deg(np.arctan((CG['init_y'] + CG['vy']*5)/(CG['init_x'] + CG['vx']*5)))
        else: 
            alpha = np.rad2deg(np.arctan(10**9))
        
        l_vel_CG.data_source.data['x'] = np.linspace(CG['init_x'], CG['init_x'] + CG['vx']*5, 10)
        l_vel_CG.data_source.data['y'] = np.linspace(CG['init_y'], CG['init_y'] + CG['vy']*5, 10)

        CG_arrow.data_source.data['x'] = [CG['init_x'] + CG['vx']*5] 
        CG_arrow.data_source.data['y'] = [CG['init_y'] + CG['vy']*5]
        CG_arrow.data_source.data['angle'] = [-alpha]
        
        
        
        
    slider_Ax.on_change('value', refresh_source)
    slider_Bx.on_change('value', refresh_source)
    slider_Ay.on_change('value', refresh_source)
    slider_By.on_change('value', refresh_source)
    slider_Av.on_change('value', refresh_source)
    slider_Bv.on_change('value', refresh_source)
    slider_A_alpha.on_change('value', refresh_source)
    slider_B_alpha.on_change('value', refresh_source)
    slider_Ar.on_change('value', refresh_source)
    slider_Br.on_change('value', refresh_source)
    
    
    def play_animation(event):
        l_vel_CG.visible = False
        CG_arrow.visible=False
        
        x_a = slider_Ax.value
        x_b = slider_Bx.value
        
        y_a = slider_Ay.value
        y_b = slider_By.value
        
        v_a = slider_Av.value
        v_b = slider_Bv.value
        
        alpha_a = slider_A_alpha.value
        alpha_b = slider_B_alpha.value
        
        rad_a = slider_Ar.value
        rad_b = slider_Br.value

        A, B, CG = initialize(T, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b) 
        update_values_time(A, B, CG, dt, scenario)

        t = 0
        idx = 0
        images = []
        sleep(0.1)
        
        while t < A['t']:
            # mettre à jour tous les paramètres pour chaque dt
            source_A.data = {'x':[A['traj'][idx,0]], 'y':[A['traj'][idx,1]], 'radius':[A['radius']]}
            source_B.data = {'x':[B['traj'][idx,0]], 'y':[B['traj'][idx,1]], 'radius':[B['radius']]}
            source_CG.data = {'x': [CG['traj'][idx, 0]], 'y': [CG['traj'][idx, 1]]}

            t += dt
            idx += 1
            
    def record(event):
        l_vel_CG.visible = False
        CG_arrow.visible=False
    
        with imageio.get_writer(file_name.value+'.mov', mode='I') as writer:
            x_a = slider_Ax.value
            x_b = slider_Bx.value

            y_a = slider_Ay.value
            y_b = slider_By.value

            v_a = slider_Av.value
            v_b = slider_Bv.value

            alpha_a = slider_A_alpha.value
            alpha_b = slider_B_alpha.value

            rad_a = slider_Ar.value
            rad_b = slider_Br.value

            A, B, CG = initialize(T, x_a, x_b, y_a, y_b, v_a, v_b, alpha_a, alpha_b, rad_a, rad_b) 
            update_values_time(A, B, CG, dt, scenario)

            t = 0
            idx = 0
            images = []
            while t < A['t']:
                # mettre à jour tous les paramètres pour chaque dt
                print('\r Generating Video. {:0.2f} % '.format(100*(t+10)/A['t']), end="")
                source_A.data = {'x':[A['traj'][idx,0]], 'y':[A['traj'][idx,1]], 'radius':[A['radius']]}
                source_B.data = {'x':[B['traj'][idx,0]], 'y':[B['traj'][idx,1]], 'radius':[B['radius']]}

                source_CG.data = {'x': [CG['traj'][idx, 0]], 'y': [CG['traj'][idx, 1]]}
                
                t += dt*10
                idx += 10

                fn = "plot_" + str(idx) + ".png"
                export_png(p, filename = fn)
                writer.append_data(imageio.imread(fn))
                os.remove(fn)

    play.on_event(ButtonClick, play_animation)
    export.on_event(ButtonClick, record)
    

    layout = column(
        row(p),
        row(slider_Ax, slider_Bx),\
        row(slider_Ay, slider_By),\
        row(slider_Ar, slider_Br),\
        row(slider_Av, slider_Bv),\
        row(slider_A_alpha, slider_B_alpha), \
        row(play, export),
        row(file_name)
    )

    
    doc.add_root(layout)



def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    
    base_url = os.environ['EXTERNAL_URL']
    host = urllib.parse.urlparse(base_url).netloc

    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    return full_url



def show_document(doc):
    servers = list(notebookapp.list_running_servers())[0]
    if servers['hostname'] == 'localhost':
        show(doc) 
    else:
        show(doc, notebook_url=remote_jupyter_proxy_url)


        
        
show_document(modify_doc)    